<a href="https://colab.research.google.com/github/NicolasChagnet/pokemon-team-optimization/blob/main/generate_improved_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generate improved dataset

In [1]:
%load_ext autoreload
%autoreload 2

import sys; sys.path.insert(0, '.')
from src import config
import pandas as pd
import pokebase

In this notebook, we will first download the [original dataset](https://www.kaggle.com/datasets/rounakbanik/pokemon) from Kaggle. Then we will improve this dataset before saving it for further use.

## Starters and pseudo-legendaries

In [3]:
df = pd.read_csv("data/pokemon.csv")
df = df.drop(
    [
        "abilities",
        "attack",
        "base_egg_steps",
        "base_happiness",
        "capture_rate",
        "classfication",
        "defense",
        "experience_growth",
        "height_m",
        "hp",
        "japanese_name",
        "percentage_male",
        "sp_attack",
        "sp_defense",
        "speed",
        "weight_kg",
    ],
    axis=1,
)
df["name"] = df["name"].str.lower()
df["is_starter"] = df["name"].isin(config.list_starters_final_stage).astype(int)
df["is_pseudo_legendary"] = df["name"].isin(config.list_pseudo_legendaries).astype(int)
df["is_fossil"] = df["name"].isin(config.list_fossils).astype(int)

df.to_csv("data/pokemon_starter_pl.csv", index=False)